Free subduction experiment in which a linear viscous slab sinks into a linear viscous mantle. The upper boundary condition is either free slip (left) or free surface (right). Clearly, the upper boundary condition has a significant effect on subduction dynamics, and only the free surface case is consistent with results of laboratory experiments. The average time step for the free surface case is significantly smaller than for the free surface case. Model parameters employed are: the slab has a density of?=3250kg/m3 and a viscosity of ?=1023 Pa s, whereas the mantle has ?=3200kg/m3, ?=1020 Pa s, the initial slab was 50km thick, 1500km long, and has a 125km long notch that initially subducted with an angle of 34◦. Free slip conditions are applied on the lower and side boundaries, and gravitational acceleration was 9.81 m/s2.
of∼800,000

In [ ]:
import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis

## Scaling

In [ ]:
u = GEO.UnitRegistry
half_rate = 1.0 * u.centimeter / u.year
model_length = 2000 * u.km
bodyforce = 3200 * u.kg / u.m**3 * 9.81 * u.m / u.s**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

## Model Geometry

In [ ]:
gravity_vector = (0.0, -9.81 * u.m / u.s**2)
Model = GEO.Model(elementRes=(128, 32), 
                  elementType="Q2/dPc1",
                  minCoord=(0. * u.km, -600. * u.km), 
                  maxCoord=(2000. * u.km, 0. * u.km), 
                  gravity=gravity_vector)
Model.outputDir="outputs_Kaus2010_FS_FreeSurf"

In [ ]:
import underworld.function as fn
import numpy as np

In [ ]:
angle = 34.0 * u.degree
dx1 = np.cos(angle) * 125. * u.kilometer
dy1 = np.sin(angle) * 125. * u.kilometer 
dx2 = np.sin(angle) * 50. * u.kilometer
dy2 = np.cos(angle) * 50. * u.kilometer

polygon1 = GEO.shapes.Polygon([(500. * u.kilometer, 0. * u.kilometer),
                              (500. * u.kilometer - dx1, 0. * u.kilometer - dy1),
                              (500. * u.kilometer - dx1 + dx2, 0. * u.kilometer - dy1 - dy2),
                              (500. * u.kilometer + dx2, 0. * u.kilometer - dy2)])

polygon2 = GEO.shapes.Polygon([(500. * u.kilometer, 0. * u.kilometer),
                              (500. * u.kilometer, -50. * u.kilometer),
                              (2000. * u.kilometer , -50. * u.kilometer),
                              (2000. * u.kilometer, 0. * u.kilometer)])

polygon = polygon1 | polygon2

In [ ]:
Mantle = Model.add_material(name="Mantle", shape=GEO.shapes.Layer(top=0.*u.kilometer, bottom=Model.bottom))
Lithosphere = Model.add_material(name="Lithosphere", shape=polygon)

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.swarm, Model.materialField)
Fig.show()

In [ ]:
x1, y1 = GEO.nd(500. * u.km - dx1), GEO.nd(-dy1)
x2, y2 = GEO.nd(500. * u.km - dx1 + dx2/2.0), GEO.nd(-dy1 - dy2/2.0)
x3, y3 = GEO.nd(500. * u.km - dx1 + dx2), GEO.nd(-dy1 - dy2)

coords = np.ndarray((3, 2))
coords[:, 0] = np.array([x1, x2, x3])
coords[:, 1] = np.array([y1, y2, y3])

Model.add_passive_tracers(name="tip", vertices=coords)

In [ ]:
from UWGeodynamics import visualisation as vis
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.tip_tracers, pointSize=5.0)
Fig.Points(Model.swarm, Model.materialField, fn_size=3.0)
Fig.show()

In [ ]:
Mantle.viscosity = 1e20 * u.pascal * u.second
Lithosphere.viscosity = 1e23 * u.pascal * u.second

Mantle.density = 3200. * u.kilogram / u.metre**3
Lithosphere.density = 3250. * u.kilogram / u.metre**3

In [ ]:
Model.set_velocityBCs(left=[0., None],
                      right=[0., None],
                      bottom=[None, 0.],
                      top=[None, None],
                      order_wall_conditions=[ "top", "left", "right", "bottom"])

In [ ]:
Model.freeSurface = True
Model.fssa_factor = 1.0

In [ ]:
Model.init_model(pressure="lithostatic")

In [ ]:
Model.solver.set_inner_method("mumps")
Model.solver.set_penalty(1e6)
GEO.rcParams["initial.nonlinear.tolerance"] = 1e-4

outputss=['pressureField',
         'strainRateField',
         'velocityField',
          'projTimeField',
           'projMaterialField',
         'projViscosityField',
         'projStressField',
         'projDensityField',
         'projStressTensor',]
GEO.rcParams['default.outputs']=outputss

In [ ]:
Model.run_for(22 * u.megayears, checkpoint_interval=0.5 * u.megayears,dt=5000*u.year)

In [ ]:
#Model.run_for(nstep=220, checkpoint_interval=20, dt=5000*u.year)

In [ ]:
#Model.run_for(nstep=4400, checkpoint_interval=200, dt=5000*u.year)

In [ ]:
#Model.run_for(nstep=588, checkpoint_interval=1, dt=500000*u.year)

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Surface(Model.mesh, Model.strainRateField, logScale=False, colours="coolwarm")
Fig.VectorArrows(Model.mesh, Model.velocityField)
Fig.show()

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.swarm, Model.materialField)
Fig.VectorArrows(Model.mesh, Model.velocityField)
Fig.show()